# ReadMe
### Esse script cria os schemas, tabelas e funções que serão utilizadas no próximo script para a tabela Servidor

In [1]:
import pandas as pd
import psycopg2

# Comandos para instalar o psycopg2
* Comando para instalar o psycopg2

conda install -c anaconda psycopg2

## Conexão com o Banco de Dados

In [2]:
try:
    conn = psycopg2.connect(dbname='pagamento_hibrido_db', user='dmining', host='texas.intranet', password='#@dmining#@')
except:
    print("I am unable to connect to the database")

In [3]:
cur = conn.cursor()
cur.execute("""

CREATE SCHEMA servidores
  AUTHORIZATION dmining;  
  
-- Órgão Superior
CREATE TABLE IF NOT EXISTS servidores.orgao_superior (
cod_orgsup VARCHAR PRIMARY KEY,
nome_orgao_superior VARCHAR
);

CREATE OR REPLACE FUNCTION servidores.duplicate_key_org_superior() returns trigger as $$
BEGIN
if(select count(*) from servidores.orgao_superior where cod_orgsup = NEW.cod_orgsup) > 0 then
     RETURN null;
  end if;
  RETURN NEW;
END;
$$ LANGUAGE plpgsql;

CREATE TRIGGER ignore_dup_org_superior
BEFORE INSERT ON servidores.orgao_superior
FOR EACH ROW
EXECUTE PROCEDURE servidores.duplicate_key_org_superior();


-- Órgão Subordinado
CREATE TABLE IF NOT EXISTS servidores.orgao_subordinado (
cod_org VARCHAR PRIMARY KEY,
nome_orgao_subordinado VARCHAR
);

CREATE OR REPLACE FUNCTION servidores.duplicate_key_org_subordinado() returns trigger as $$
BEGIN
if(select count(*) from servidores.orgao_subordinado where cod_org = NEW.cod_org) > 0 then
     RETURN null;
  end if;
  RETURN NEW;
END;
$$ LANGUAGE plpgsql;

CREATE TRIGGER ignore_dup_org_subordinado
BEFORE INSERT ON servidores.orgao_subordinado
FOR EACH ROW
EXECUTE PROCEDURE servidores.duplicate_key_org_subordinado();


-- Unidade Órgão
CREATE TABLE IF NOT EXISTS servidores.unidade_orgao (
cod_uorg VARCHAR PRIMARY KEY,
nome_unidade_orgao VARCHAR
);

CREATE OR REPLACE FUNCTION servidores.duplicate_key_uorg() returns trigger as $$
BEGIN
if(select count(*) from servidores.unidade_orgao where cod_uorg = NEW.cod_uorg) > 0 then
     RETURN null;
  end if;
  RETURN NEW;
END;
$$ LANGUAGE plpgsql;

CREATE TRIGGER ignore_dup_uorg
BEFORE INSERT ON servidores.unidade_orgao
FOR EACH ROW
EXECUTE PROCEDURE servidores.duplicate_key_uorg();

-- Remuneração
CREATE TABLE IF NOT EXISTS servidores.remuneracao(
ano integer, 
  mes integer,
  id_servidor_portal bigint, 
  cpf varchar, 
  nome varchar, 
  remuneracao_basica_bruta_brl numeric(12,2), 
  remuneracao_basica_bruta_url numeric(12,2), 
  data integer
);

-- Cargo
CREATE TABLE IF NOT EXISTS servidores.cargo(
id_cargo serial primary key,
  descricao_cargo varchar, 
  classe_cargo varchar, 
  referencia_cargo varchar, 
  padrao_cargo varchar, 
  nivel_cargo varchar
);

CREATE OR REPLACE FUNCTION servidores.duplicate_key_cargo() returns trigger as $$
BEGIN
if(select count(*) from servidores.cargo where descricao_cargo = NEW.descricao_cargo AND classe_cargo = NEW.classe_cargo AND
referencia_cargo = NEW.referencia_cargo AND padrao_cargo = NEW.padrao_cargo AND 
nivel_cargo = NEW.nivel_cargo) > 0 then
     RETURN null;
  end if;
  RETURN NEW;
END;
$$ LANGUAGE plpgsql;

CREATE TRIGGER ignore_dup_cargo
BEFORE INSERT ON servidores.cargo
FOR EACH ROW
EXECUTE PROCEDURE servidores.duplicate_key_cargo();

-- Função
CREATE TABLE IF NOT EXISTS servidores.funcao(
id_funcao serial primary key,
  sigla_funcao varchar, 
  nivel_funcao varchar, 
  funcao varchar
);

CREATE OR REPLACE FUNCTION servidores.duplicate_key_funcao() returns trigger as $$
BEGIN
if(select count(*) from servidores.funcao where sigla_funcao = NEW.sigla_funcao AND nivel_funcao = NEW.nivel_funcao
AND funcao = NEW.funcao) > 0 then
     RETURN null;
  end if;
  RETURN NEW;
END;
$$ LANGUAGE plpgsql;

CREATE TRIGGER ignore_dup_funcao
BEFORE INSERT ON servidores.funcao
FOR EACH ROW
EXECUTE PROCEDURE servidores.duplicate_key_funcao();

-- Atividade
CREATE TABLE IF NOT EXISTS servidores.atividade(
id_atividade serial primary key,
  codigo_atividade varchar, 
  atividade varchar
);

CREATE OR REPLACE FUNCTION servidores.duplicate_key_atividade() returns trigger as $$
BEGIN
if(select count(*) from servidores.atividade where codigo_atividade = NEW.codigo_atividade 
AND atividade = NEW.atividade) > 0 then
     RETURN null;
  end if;
  RETURN NEW;
END;
$$ LANGUAGE plpgsql;

CREATE TRIGGER ignore_dup_atividade
BEFORE INSERT ON servidores.atividade
FOR EACH ROW
EXECUTE PROCEDURE servidores.duplicate_key_atividade();


-- Vinculo
CREATE TABLE IF NOT EXISTS servidores.vinculo(
id_vinculo serial primary key,
  tipo_vinculo varchar,
  situacao_vinculo varchar
);

CREATE OR REPLACE FUNCTION servidores.duplicate_key_vinculo() returns trigger as $$
BEGIN
if(select count(*) from servidores.vinculo where tipo_vinculo = NEW.tipo_vinculo 
AND situacao_vinculo = NEW.situacao_vinculo) > 0 then
     RETURN null;
  end if;
  RETURN NEW;
END;
$$ LANGUAGE plpgsql;

CREATE TRIGGER ignore_dup_vinculo
BEFORE INSERT ON servidores.vinculo
FOR EACH ROW
EXECUTE PROCEDURE servidores.duplicate_key_vinculo();

CREATE TABLE IF NOT EXISTS servidores.fato_cargo_funcao_remuneracao(
  tipo char(1),
  id_servidor_portal bigint, 
  id_cargo integer,
  id_funcao integer,
  id_atividade integer,
  id_vinculo integer,
  cod_uorg_lotacao varchar, 
  cod_org_lotacao varchar, 
  cod_orgsup_lotacao varchar, 
  cod_uorg_exercicio varchar, 
  cod_org_exercicio varchar, 
  cod_orgsup_exercicio varchar, 
  ano integer, 
  mes integer, 
  remuneracao_basica_bruta_brl numeric(12,2), 
  remuneracao_basica_bruta_url numeric(12,2), 
  data integer,
  FOREIGN KEY(cod_uorg_lotacao) REFERENCES servidores.unidade_orgao (cod_uorg),
  FOREIGN KEY(cod_uorg_exercicio) REFERENCES servidores.unidade_orgao (cod_uorg),
  FOREIGN KEY(cod_org_lotacao) REFERENCES servidores.orgao_subordinado (cod_org),
  FOREIGN KEY(cod_org_exercicio) REFERENCES servidores.orgao_subordinado (cod_org),
  FOREIGN KEY(cod_orgsup_lotacao) REFERENCES servidores.orgao_superior (cod_orgsup),
  FOREIGN KEY(cod_orgsup_exercicio) REFERENCES servidores.orgao_superior (cod_orgsup),
  FOREIGN KEY(id_cargo) REFERENCES servidores.cargo (id_cargo),
  FOREIGN KEY(id_funcao) REFERENCES servidores.funcao (id_funcao),
  FOREIGN KEY(id_atividade) REFERENCES servidores.atividade (id_atividade),
  FOREIGN KEY(id_vinculo) REFERENCES servidores.vinculo (id_vinculo)
);
  
CREATE TABLE IF NOT EXISTS servidores.cadastro(
id_servidor_portal bigint, 
  nome varchar, 
  cpf varchar, 
  matricula varchar, 
  id_cargo integer,
  id_funcao integer,
  id_atividade integer,
  id_vinculo integer,
  cod_uorg_lotacao varchar, 
  cod_org_lotacao varchar, 
  cod_orgsup_lotacao varchar, 
  cod_uorg_exercicio varchar, 
  cod_org_exercicio varchar, 
  cod_orgsup_exercicio varchar, 
  data_inicio_afastamento varchar, 
  data_termino_afastamento varchar, 
  regime_juridico varchar, 
  jornada_de_trabalho varchar, 
  uf_exercicio varchar,
  UNIQUE (id_servidor_portal, id_cargo, id_funcao, id_atividade, id_vinculo,
  cod_uorg_exercicio, cod_org_exercicio, cod_orgsup_exercicio),
  FOREIGN KEY(cod_uorg_lotacao) REFERENCES servidores.unidade_orgao (cod_uorg),
  FOREIGN KEY(cod_uorg_exercicio) REFERENCES servidores.unidade_orgao (cod_uorg),
  FOREIGN KEY(cod_org_lotacao) REFERENCES servidores.orgao_subordinado (cod_org),
  FOREIGN KEY(cod_org_exercicio) REFERENCES servidores.orgao_subordinado (cod_org),
  FOREIGN KEY(cod_orgsup_lotacao) REFERENCES servidores.orgao_superior (cod_orgsup),
  FOREIGN KEY(cod_orgsup_exercicio) REFERENCES servidores.orgao_superior (cod_orgsup),
  FOREIGN KEY(id_cargo) REFERENCES servidores.cargo (id_cargo),
  FOREIGN KEY(id_funcao) REFERENCES servidores.funcao (id_funcao),
  FOREIGN KEY(id_atividade) REFERENCES servidores.atividade (id_atividade),
  FOREIGN KEY(id_vinculo) REFERENCES servidores.vinculo (id_vinculo)
);
  
--Trigger to avoid duplicate keys during COPY or INSERT
CREATE OR REPLACE FUNCTION servidores.duplicate_key_cadastro() returns trigger as $$
BEGIN
if(select count(*) from servidores.cadastro where id_servidor_portal = NEW.id_servidor_portal AND 
id_cargo = NEW.id_cargo AND id_funcao = NEW.id_funcao AND id_atividade = NEW.id_atividade AND 
id_vinculo = NEW.id_vinculo AND cod_uorg_exercicio = NEW.cod_uorg_exercicio AND 
cod_org_exercicio = NEW.cod_org_exercicio AND cod_orgsup_exercicio = NEW.cod_orgsup_exercicio) > 0 then
     RETURN null;
  end if;
  RETURN NEW;
END;
$$ LANGUAGE plpgsql;

CREATE TRIGGER ignore_dup
BEFORE INSERT ON servidores.cadastro
FOR EACH ROW
EXECUTE PROCEDURE servidores.duplicate_key_cadastro();
""")

In [4]:
conn.commit()